In [ ]:
%pip install bleak

In [ ]:
import asyncio
from bleak import BleakScanner, BleakClient

In [ ]:
print("Scanning for BLE devices…")
devices = await BleakScanner.discover(timeout=5.0)

if not devices:
    print("No devices found.")
else:
    for d in devices:
        print(f"Name: {d.name!r}, Address: {d.address}")


In [ ]:
TARGET_NAME = "LEDPeripheral"
CHAR_UUID   = "19B10001-E8F2-537E-4F6C-D104768A1214"

def read_flatworm_direction() -> int:
    # Replace with real signal‑processing
    return 0

async def main():
    client = None
    while True:
        if client is None or not client.is_connected:
            print("Scanning…")
            devices = await BleakScanner.discover(timeout=5.0)
            addr = next((d.address for d in devices if d.name == TARGET_NAME), None)
            if not addr:
                print(f"{TARGET_NAME} not found")
                await asyncio.sleep(2); continue

            client = BleakClient(addr)
            try:
                await client.connect()
                print("Connected")
            except Exception as e:
                print("Connect failed:", e)
                client = None
                await asyncio.sleep(2)
                continue

        # once connected:
        direction = read_flatworm_direction()
        if 0 <= direction <= 6:
            try:
                await client.write_gatt_char(CHAR_UUID, bytes([direction]))
                print("Sent:", direction)
            except Exception as e:
                print("Write failed:", e)

        # handle drop
        if client and not client.is_connected:
            print("Disconnected")
            await client.disconnect()
            client = None

        await asyncio.sleep(0.5)

# **In Jupyter, just do:**
await main()

